### Azpn : Azure VPN Free (For Students Plan)
_Premise: Azure student plan gives students a good 100 dollars starter credits yearly, we can use this quota to provide ourself with very cheap but useful VPN Server_

__Whatchu Need?__

A student email id (with .edu ending in last) and verified azure student account, If you are a student and signup to azure, It gives you 100\\$ per year until you graduate! So simply exploiting this loophole and deploying server using those 100\\$ of free creds is what Azpn do! Most people would never even use Azure much, if you don't mind using some of your credits for a free secure VPN then continue reading! :)

## Helper functions

In [21]:
import sys
import subprocess

def execute(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    # Poll process for new output until finished
    r = ""
    while True:
        nextline = process.stdout.readline()
        if not nextline and process.poll() is not None:
            break
        r += nextline.decode("utf-8") .strip() + " "
        sys.stdout.write(nextline)
        sys.stdout.flush()

    output = process.communicate()[0]
    exitCode = process.returncode

    if (exitCode == 0):
        return r
    else:
        print("error?")
        
        
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
def generate_temp_password(length):
    if not isinstance(length, int) or length < 8:
        raise ValueError("temp password must have positive length")

    chars = "@ABCDEFGHJKLMNPQRSTUVWXYZ23456789abcdefghijklmnopqrstuvwxyz@"
    from os import urandom

    # original Python 2 (urandom returns str)
    # return "".join(chars[ord(c) % len(chars)] for c in urandom(length))

    # Python 3 (urandom returns bytes)
    return "".join(chars[c % len(chars)] for c in urandom(length))

def downloadFile(fname):
    stdin,stdout,stderr=ssh.exec_command("sudo cat /home/%s/%s"%(ADMIN_USERNAME, fname))
    outlines=stdout.readlines()
    resp=''.join(outlines)
    print(resp)

    with open(fname, 'w') as file:
        file.write(resp)

### Global Configs

In [22]:
import uuid
import secrets
import string

RESOURCE_GROUP_NAME = "azpn_group"
VM_INSTANCE_NAME = "azpn_instance001"
MACHINE_NAME = "Standard_B1ls" # Cheapest, Scale
ADMIN_USERNAME = "azpn"
ADMIN_PASSWORD = generate_temp_password(48) # Unique one time password

In [23]:
printmd("__Admin password generated, please save this somewhere or you risk losing access to your virtual machine__")
printmd("PASSWORD IS __%s__"%(ADMIN_PASSWORD))

__Admin password generated, please save this somewhere or you risk losing access to your virtual machine__

PASSWORD IS __UzVcnmo8NJBkLFocT5TyeMPUK64S5jB7M4ESXcFtAbnvtif5__

### Start
_login azure-cli_

In [11]:
x = execute("az login")
if("Azure for Students" not in x):
    printmd("**Wrong account? Couldn't find azure students plan or cannot login to service, try again!**")

CloudName    HomeTenantId                          IsDefault    Name                State    TenantId
-----------  ------------------------------------  -----------  ------------------  -------  ------------------------------------
AzureCloud   611eb846-2373-4df4-9c07-cfe7dd7973bf  True         Azure for Students  Enabled  611eb846-2373-4df4-9c07-cfe7dd7973bf


__Get all resource regions (ie VPN Region)__

In [19]:
locs = execute('az account list-locations --query "[].{DisplayName:displayName, Name:name}" -o table')

DisplayName               Name
------------------------  -------------------
East US                   eastus
East US 2                 eastus2
South Central US          southcentralus
West US 2                 westus2
West US 3                 westus3
Australia East            australiaeast
Southeast Asia            southeastasia
North Europe              northeurope
Sweden Central            swedencentral
UK South                  uksouth
West Europe               westeurope
Central US                centralus
North Central US          northcentralus
West US                   westus
South Africa North        southafricanorth
Central India             centralindia
East Asia                 eastasia
Japan East                japaneast
Jio India West            jioindiawest
Korea Central             koreacentral
Canada Central            canadacentral
France Central            francecentral
Germany West Central      germanywestcentral
Norway East               norwayeast
Switzerland Nor

__Input your preferred region__

In [ ]:
def reg():
    r = input("Enter your preferred region (default-centralindia): ") or "centralindia"
    if("            " + r + " " not in locs):
        print("Invalid/Bad region, enter again.")
        reg()
    else:
        return r

REGION = reg()
printmd("_Selected region: %s_"%(REGION))

_Create resource group for VM and other resources_

In [22]:
x = execute("az group create --name %s --location %s --tags USE=AZPN -o table"%(RESOURCE_GROUP_NAME, REGION))

Location      Name
------------  ----------
centralindia  azpn_group


In [23]:
c = """az vm create
    --resource-group %s
    --name %s
    --size %s
    --image Canonical:0001-com-ubuntu-server-focal:20_04-lts:latest
    --public-ip-sku Basic
    --public-ip-address-allocation dynamic
    --admin-username %s
    --admin-password %s
    --storage-sku Standard_LRS
    --location %s
    --data-disk-delete-option Delete
    --nic-delete-option Delete
    --data-disk-delete-option Delete
    -o json""" % (RESOURCE_GROUP_NAME, VM_INSTANCE_NAME, MACHINE_NAME,ADMIN_USERNAME, ADMIN_PASSWORD, REGION)
print(c)
x = execute(c.replace("\n",""))

az vm create
    --resource-group azpn_group
    --name azpn_instance001
    --size Standard_B1ls
    --image Canonical:0001-com-ubuntu-server-focal:20_04-lts:latest
    --public-ip-sku Basic
    --public-ip-address-allocation dynamic
    --admin-username azpn
    --admin-password EUTg85nAMMArV@7TmwdEMpvQGq@yiaw3@VtVYbKLNedlwCrK
    --storage-sku Standard_LRS
    --location centralindia
    --data-disk-delete-option Delete
    --nic-delete-option Delete
    --data-disk-delete-option Delete
    -o json
{
  "fqdns": "",
  "id": "/subscriptions/14652e31-aac4-4a7d-a6f8-32c93da1e266/resourceGroups/azpn_group/providers/Microsoft.Compute/virtualMachines/azpn_instance001",
  "location": "centralindia",
  "macAddress": "60-45-BD-AC-15-C0",
  "powerState": "VM running",
  "privateIpAddress": "10.0.0.4",
  "publicIpAddress": "20.207.202.196",
  "resourceGroup": "azpn_group",
  "zones": ""
}


In [24]:
import json
vm_data = json.loads(x)
if('publicIpAddress' not in vm_data):
    printmd("__Cannot find public ip address of vm, creation problem perhaps?__")

In [25]:
print(vm_data)

{'fqdns': '', 'id': '/subscriptions/14652e31-aac4-4a7d-a6f8-32c93da1e266/resourceGroups/azpn_group/providers/Microsoft.Compute/virtualMachines/azpn_instance001', 'location': 'centralindia', 'macAddress': '60-45-BD-AC-15-C0', 'powerState': 'VM running', 'privateIpAddress': '10.0.0.4', 'publicIpAddress': '20.207.202.196', 'resourceGroup': 'azpn_group', 'zones': ''}


In [16]:
SSH_IP = vm_data["publicIpAddress"]
# SSH_IP = "20.207.202.196"
# ADMIN_PASSWORD = "passs"
SSH_PORT = 22

printmd("__Connecting to %s:%s__"%(SSH_IP, SSH_PORT))

import paramiko
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(SSH_IP,SSH_PORT,ADMIN_USERNAME,ADMIN_PASSWORD)

printmd("__Connected! Upgrading packages.. (might take a while)__")

__Connecting to 20.207.202.196:22__

__Connected! Upgrading packages.. (might take a while)__

In [6]:
stdin,stdout,stderr=ssh.exec_command("sudo apt-get update -y && sudo apt-get upgrade -y")
outlines=stdout.readlines()
resp=''.join(outlines)
print(resp)

Reading package lists...



_Download openvpn-install.sh file_

In [29]:
_, _, _ = ssh.exec_command("curl -O https://raw.githubusercontent.com/angristan/openvpn-install/master/openvpn-install.sh && chmod +x openvpn-install.sh")

In [17]:
stdin,stdout,stderr=ssh.exec_command("sudo AUTO_INSTALL=y PORT_CHOICE=2 PORT=443 PROTOCOL_CHOICE=2 ./openvpn-install.sh")
outlines=stdout.readlines()
opnout=''.join(outlines)
print(opnout)

Welcome to the OpenVPN installer!
The git repository is available at: https://github.com/angristan/openvpn-install

I need to ask you a few questions before starting the setup.
You can leave the default options and just press enter if you are ok with them.

I need to know the IPv4 address of the network interface you want OpenVPN listening to.
Unless your server is behind NAT, it should be your public IPv4 address.

It seems this server is behind NAT. What is its public IPv4 address or hostname?
We need it for the clients to connect to the server.

Checking for IPv6 connectivity...

Your host does not appear to have IPv6 connectivity.


What port do you want OpenVPN to listen to?
   1) Default: 1194
   2) Custom
   3) Random [49152-65535]

What protocol do you want OpenVPN to use?
UDP is faster. Unless it is not available, you shouldn't use TCP.
   1) UDP
   2) TCP

What DNS resolvers do you want to use with the VPN?
   1) Current system resolvers (from /etc/resolv.conf)
   2) Self-hos

In [18]:
if not "The configuration file has been written to" in opnout:
    printmd("__Error in installation or whaat?__")
    
printmd("__Downloading client.ovpn for connection!__")
downloadFile("client.ovpn")    
printmd("__Downloaded! Enjoy!! :P__")

__Downloading client.ovpn for connection!__

client
proto tcp-client
remote 20.207.202.196 443
dev tun
resolv-retry infinite
nobind
persist-key
persist-tun
remote-cert-tls server
verify-x509-name server_ORN1JdbJCIraWFNb name
auth SHA256
auth-nocache
cipher AES-128-GCM
tls-client
tls-version-min 1.2
tls-cipher TLS-ECDHE-ECDSA-WITH-AES-128-GCM-SHA256
ignore-unknown-option block-outside-dns
setenv opt block-outside-dns # Prevent Windows 10 DNS leak
verb 3
<ca>
-----BEGIN CERTIFICATE-----
MIIB2DCCAX2gAwIBAgIUJgC5b/Zq4o4zEpIc/lNz2Ep74hMwCgYIKoZIzj0EAwIw
HjEcMBoGA1UEAwwTY25fUkVZblc3YTBvNXJIb0ZBdzAeFw0yMjAyMjQwNzUwNDBa
Fw0zMjAyMjIwNzUwNDBaMB4xHDAaBgNVBAMME2NuX1JFWW5XN2EwbzVySG9GQXcw
WTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAARmV9M0NgaI+HgNVQlR8q/bee2dgskz
sJquFzXjAYiKVXWkQHFbmvhn4lhxUxFYwhbpB0Ehkh77lVbbg1giecw+o4GYMIGV
MB0GA1UdDgQWBBRVqZDayRt7x4UkNfAZAKJacULViDBZBgNVHSMEUjBQgBRVqZDa
yRt7x4UkNfAZAKJacULViKEipCAwHjEcMBoGA1UEAwwTY25fUkVZblc3YTBvNXJI
b0ZBd4IUJgC5b/Zq4o4zEpIc/lNz2Ep74hMwDAYDVR0TBAUwAwEB/zALBgNVHQ8E
BAMCAQYwCgYIKoZIzj0EAwIDSQAwRgIhAPH

__Downloaded! Enjoy!! :P__

### Lastly open 443 port for our VPN

In [32]:
x = execute("az vm open-port --resource-group %s --name %s --port 443"%(RESOURCE_GROUP_NAME, VM_INSTANCE_NAME))

Location      Name                 ProvisioningState    ResourceGroup    ResourceGuid
------------  -------------------  -------------------  ---------------  ------------------------------------
centralindia  azpn_instance001NSG  Succeeded            azpn_group       83acf083-0c94-4cda-9cae-02971b6b22b8


Socket exception: An existing connection was forcibly closed by the remote host (10054)


## SUCCESS

If you reached here, it mean's you have successfully deployed a working VPN on your azure server! Now what? Simply Install latest openvpn client and connect to profile _client.ovpn_(in same directory)!

_Also, if you ever want to save your creds by deleting everything azpn has done until now, then execute commands in next catalog (3), namely azpn_remove_all, It will undo any changes made by this script!_

# Bonus
What if you want to share your created VPN with multiple users? Well don't worry, this bonus section is for those people wanting to generate more than one profiles for openvpn, So as to share it with their friends without them needing to install anything!

In [20]:
def no():
    try:
        x = int(input("How many profiles to generate?"))
        return x
    except:
        print("Invalid input, enter a number!")
        return no()
    
PROFILE_NO = no()
for i in range(PROFILE_NO):
    confName = "client{}".format(i+1)
    cmd = 'sudo MENU_OPTION="1" CLIENT="%s" PASS="1" ./openvpn-install.sh'%(confName)
    print(cmd)
    stdin,stdout,stderr=ssh.exec_command(cmd)
    outlines=stdout.readlines()
    resp=''.join(outlines)
    print(resp)
    downloadFile("%s.ovpn"%(confName))    

How many profiles to generate? 2


sudo MENU_OPTION="1" CLIENT="client1" PASS="1" ./openvpn-install.sh
Welcome to OpenVPN-install!
The git repository is available at: https://github.com/angristan/openvpn-install

It looks like OpenVPN is already installed.

What do you want to do?
   1) Add a new user
   2) Revoke existing user
   3) Remove OpenVPN
   4) Exit

Tell me a name for the client.
The name must consist of alphanumeric character. It may also include an underscore or a dash.

Do you want to protect the configuration file with a password?
(e.g. encrypt the private key with a password)
   1) Add a passwordless client
   2) Use a password for the client

The specified client CN was already found in easy-rsa, please choose another name.

client
proto tcp-client
remote 20.207.202.196 443
dev tun
resolv-retry infinite
nobind
persist-key
persist-tun
remote-cert-tls server
verify-x509-name server_ORN1JdbJCIraWFNb name
auth SHA256
auth-nocache
cipher AES-128-GCM
tls-client
tls-version-min 1.2
tls-cipher TLS-ECDHE-ECDSA-W